## Scraping Multiple Pages - Albuquerque

#### Importing Libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import urllib.parse
import pandas as pd

#### Scraping the data

In [2]:
address = []
bedrooms = []
bathrooms = []
sqft = []
year_built = []
parking = []
pool = []
price = []

# Url part I
url_1 = 'https://www.trulia.com'

for i in range(1, 26):
    
    #website
    www = 'https://www.trulia.com/NM/Albuquerque/' +str(i) +'_p/'
    
    #requests
    response = requests.get(www)
    
    # soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    #result container
    result_container = soup.find_all('li', {'class':'SearchResultsList__WideCell-b7y9ki-2'})
    
    #update results
    results_update = []
    
    # only results with attribute 'data-testeid'
    for r in result_container:
        if r.has_attr('data-testid'):
            results_update.append(r)
        
    #relative url
    relative_url = []
    
    #loop through results
    for i in results_update:
        for j in i.find_all('div', {'data-testid':'property-card-details'}):
            relative_url.append(j.find('a').get('href'))
            
    #empty list url_joined
    url_joined = []

    for i in relative_url:
        url_joined.append(urllib.parse.urljoin(url_1, i))
        
       
    # loop through all joined links
    for i in url_joined:
        response = requests.get(i)

        #create soup object
        soup = BeautifulSoup(response.content, 'html.parser')

        #address
        try:
            address.append(soup.find('span', {'data-testid':'home-details-summary-headline'}).get_text())
        except:
            address.append('')

        #bedrooms
        try:
            bedrooms.append(soup.find('li', {'data-testid':'bed'}).get_text())
        except:
            bedrooms.append('')

        #bathrooms
        try:
            bathrooms.append(soup.find('li', {'data-testid':'bath'}).get_text())
        except:
            bathrooms.append('')

        #sqft
        try:
            sqft.append(soup.find('li', {'data-testid':'floor'}).get_text())
        except:
            sqft.append('')

        #year built
        try:
            year_built.append(soup.find('div', string='Year Built').findNext('div').get_text())
        except:
            year_built.append('')

         #parking
        try:
            parking.append(soup.find('div', string='Parking').findNext('div').get_text())
        except:
            parking.append('')

         #pool
        try:
            pool.append(soup.find('div', string='Pool').findNext('div').get_text())
        except:
            pool.append('')

         #price
        try:
            price.append(soup.find('h3', {'data-testid':'on-market-price-details'}).get_text())
        except:
            price.append('')

        #creating a dictionary with the results
        output = {'Address': address, 'Bedrooms':bedrooms, 'Bathrooms': bathrooms, 'Area': sqft, 'Year Built': year_built,
                 'Parking': parking, 'Pool': pool, 'Price': price}

In [3]:
df = pd.DataFrame(output)
df

,Address,Bedrooms,Bathrooms,Area,Year Built,Parking,Pool,Price
0,6716 Carney Ave NW,3 Beds,2 Baths,"1,687 sqft",1992,2 Car Garage,No,"$250,000"
1,7429 Gila Rd NE,4 Beds,2 Baths,"2,245 sqft",1973,Garage,No,"$445,990"
2,9919 Brunswick Pl NW,4 Beds,3 Baths,"2,168 sqft",2004,2 Car Garage,No,"$350,000"
3,4413 Delamar Ave NE,3 Beds,2 Baths,"1,867 sqft",1959,2 Carport Spaces,No,"$255,000"
4,4618 Inspiration Dr SE,4 Beds,2 Baths,"1,973 sqft",1951,No Info,No,"$280,000"
...,...,...,...,...,...,...,...,...
995,Fruta Rd NE,,,,,,,"$150,000"
996,159 Suez Ct,4 Beds,3 Baths,"2,374 sqft (on 0.37 acres)",2020,Garage,No,"$476,525"
997,1111 Golf Course Rd SE,,,,,,,"$165,000"
998,2112 Chessman Dr NE,,,,,,,"$69,000"


In [4]:
df['Location'] = 'Albuquerque'
df

,Address,Bedrooms,Bathrooms,Area,Year Built,Parking,Pool,Price,Location
0,6716 Carney Ave NW,3 Beds,2 Baths,"1,687 sqft",1992,2 Car Garage,No,"$250,000",Albuquerque
1,7429 Gila Rd NE,4 Beds,2 Baths,"2,245 sqft",1973,Garage,No,"$445,990",Albuquerque
2,9919 Brunswick Pl NW,4 Beds,3 Baths,"2,168 sqft",2004,2 Car Garage,No,"$350,000",Albuquerque
3,4413 Delamar Ave NE,3 Beds,2 Baths,"1,867 sqft",1959,2 Carport Spaces,No,"$255,000",Albuquerque
4,4618 Inspiration Dr SE,4 Beds,2 Baths,"1,973 sqft",1951,No Info,No,"$280,000",Albuquerque
...,...,...,...,...,...,...,...,...,...
995,Fruta Rd NE,,,,,,,"$150,000",Albuquerque
996,159 Suez Ct,4 Beds,3 Baths,"2,374 sqft (on 0.37 acres)",2020,Garage,No,"$476,525",Albuquerque
997,1111 Golf Course Rd SE,,,,,,,"$165,000",Albuquerque
998,2112 Chessman Dr NE,,,,,,,"$69,000",Albuquerque


In [5]:
df.to_csv('df_albuquerque.csv', index=False)